In [96]:
import pandas as pd
import bs4


In [97]:
diseases = pd.read_csv('diseases_with_desc.csv')
diseases.dropna(inplace=True)

In [98]:
subAttr = {'data-attrid': 'subtitle'}
descAttr = {'data-attrid': 'kc:/medicine/disease:description'}
secondaryDescAttr = {'data-attrid': 'kc:/medicine/disease:long description'}
rarityAttr = {'data-attrid': 'kc:/medicine/disease:location prevalence title'}
symptomAttr = {'data-attrid': 'kc:/medicine/disease:detailed symptoms'}
